Module Imports

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

TensorFlow 2.x selected.


Mounting The Drive.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Downloading Fasttext Vectors

In [0]:
#Fasttext Vectors
downloaded = drive.CreateFile({'id':"1P7Ax8KnEhih7wgYmmwqGarl837cAAkRQ"})   # replace the id with id of file you want to access
downloaded.GetContentFile('cc.zh.300.vec.gz')        # replace the file name with your file

In [0]:
#Fasttext Vectors
downloaded = drive.CreateFile({'id':"1vQrV6RDI3e2tsHspMft6zM5DYvRK4wSD"})   # replace the id with id of file you want to access
downloaded.GetContentFile('cc.en.300.vec.gz')        # replace the file name with your file

In [0]:
# importing required modules 
from zipfile import ZipFile 
  
import gzip
import shutil
with gzip.open('cc.zh.300.vec.gz', 'rb') as f_in:
    with open('file1.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [0]:
# importing required modules 
from zipfile import ZipFile 
  
import gzip
import shutil
with gzip.open('cc.en.300.vec.gz', 'rb') as f_in:
    with open('file2.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

Create Embedding Index

In [0]:
import numpy as np 
embeddings_index_zh = {}
f = open('file1.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_zh[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_zh))

Found 2000000 word vectors.


In [0]:
import numpy as np 
embeddings_index_en = {}
f = open('file2.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_en[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_en))

Found 2000000 word vectors.


Downloading the Dataset

In [0]:
#Chinese Dataset
downloaded = drive.CreateFile({'id':"1ggvUaxzL3GAITVh4fyVQQ11upVEU_7dW"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Chinese Many Things.xlsx')        # replace the file name with your file

In [0]:
#Reading the Dataset
import pandas as pd

english_data=[]
chinese_data=[]

df=pd.read_excel(r'Chinese Many Things.xlsx')
for i in range (df.shape[0]):
  english_data.append(df.iloc[i,0])
  chinese_data.append(df.iloc[i,1])

Preprocessing

In [0]:
pip install text_cleaner

In [0]:
#clean chinese 
from text_cleaner.processor.chinese import CHINESE, CHINESE_SYMBOLS_AND_PUNCTUATION
from text_cleaner import remove
from text_cleaner.processor.common import ASCII, SYMBOLS_AND_PUNCTUATION_EXTENSION, GENERAL_PUNCTUATION

def clean_zh(lines):
  lines1=[]
  for line in lines:
    line = CHINESE_SYMBOLS_AND_PUNCTUATION.remove(line)
    line = SYMBOLS_AND_PUNCTUATION_EXTENSION.remove(line)
    line = GENERAL_PUNCTUATION.remove(line)
    line = line.translate({ord('€'): None})
    line = re.sub(r'[" "]+', " ", line)
    lines1.append(line)
  return lines1

#Remove English Characters and Numbers from Chinese
import re
def escape_en_char(lines):
  lines1=[]
  en_pattern = "[a-zA-Z0-9]+"
  for line in lines:
     line = re.sub(en_pattern, ' ', line)
     line = re.sub(r'[" "]+', " ", line)
     lines1.append(line)
  return lines1

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
  
def clean_en(lines):
  lines1 = []
  for w in lines :
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z]+", " ", w)
    w = w.rstrip().strip()
    lines1.append(w)
  return lines1

In [0]:
import jieba

In [0]:
totalX = []
for i in range(len(chinese_data)):
  text=str(chinese_data[i])
  words = jieba.cut(text, cut_all=False)
  text = " ".join(words)
  totalX.append(text)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.860 seconds.
Prefix dict has been built succesfully.


In [0]:
en_clean=clean_en(english_data)
zh_clean=clean_zh(totalX)
zh_clean=escape_en_char(zh_clean)
english_clean=[]
chinese_clean=[]
for i in range(len(en_clean)):#was en_clean
  en_clean[i] = '<start> ' + str(en_clean[i]) + ' <end>'
  zh_clean[i] = '<start> ' + str(zh_clean[i]) + ' <end>'
  english_clean.append(en_clean[i])
  chinese_clean.append(zh_clean[i])

In [0]:
chinese_clean_temp=chinese_clean
english_clean_temp=english_clean
chinese_clean=[]
english_clean=[]

In [0]:
for i in range(len(chinese_clean_temp)):
  if (chinese_clean_temp[i]!=' ') and (english_clean_temp[i]!=' '):
    chinese_clean_temp[i]=chinese_clean_temp[i].translate({ord('0'): None})
    chinese_clean.append(chinese_clean_temp[i])
    english_clean.append(english_clean_temp[i])

In [0]:
def convert_list(list):
  return tuple(list)

In [0]:
en=convert_list(english_clean)
zh=convert_list(chinese_clean)

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [0]:
def tensor_and_tokeniser(inp_lang, targ_lang):
  # creating cleaned input, output pairs
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
input_tensor, target_tensor, inp_lang, targ_lang = tensor_and_tokeniser(zh, en)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [0]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

16963 16963 4241 4241


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

Create a tf.data dataset







In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

Creating Model Architecture

In [0]:
#creating embedding input matrix
embedding_dim = 300
embedding_matrix_zh = np.zeros((vocab_inp_size, embedding_dim))
for word, i in inp_lang.word_index.items() :
    embedding_vector = embeddings_index_zh.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_zh[i] = embedding_vector

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    # self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.embedding = tf.keras.layers.Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_matrix_zh],
                            input_length=None,
                            trainable=False, mask_zero = True)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

Encoder output shape: (batch size, sequence length, units) (64, 30, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
attention_layer = BahdanauAttention(10)

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 30, 1)


In [0]:
#creating output embedding matrix
embedding_matrix_en = np.zeros((vocab_tar_size, embedding_dim))
for word, i in targ_lang.word_index.items() :
    embedding_vector = embeddings_index_en.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_en[i] = embedding_vector

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    # self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.embedding = tf.keras.layers.Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_matrix_en],
                            input_length=None,
                            trainable=False, mask_zero = True)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

Decoder output shape: (batch_size, vocab size) (64, 6146)


Optimizer and the Loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

Training

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 1 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.4231
Epoch 1 Batch 100 Loss 0.4047
Epoch 1 Batch 200 Loss 0.4173
Epoch 1 Loss 0.4442
Time taken for 1 epoch 442.2458372116089 sec

Epoch 2 Batch 0 Loss 0.4299
Epoch 2 Batch 100 Loss 0.4065
Epoch 2 Batch 200 Loss 0.4192
Epoch 2 Loss 0.4069
Time taken for 1 epoch 444.7548191547394 sec

Epoch 3 Batch 0 Loss 0.3092
Epoch 3 Batch 100 Loss 0.3630
Epoch 3 Batch 200 Loss 0.4011
Epoch 3 Loss 0.3730
Time taken for 1 epoch 442.7626509666443 sec

Epoch 4 Batch 0 Loss 0.2881
Epoch 4 Batch 100 Loss 0.2981
Epoch 4 Batch 200 Loss 0.3825
Epoch 4 Loss 0.3429
Time taken for 1 epoch 443.6432423591614 sec

Epoch 5 Batch 0 Loss 0.2757


KeyboardInterrupt: ignored

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  # sentence = preprocess_sentence(sentence)
  words = jieba.cut(sentence, cut_all=False)
  sentence = " ".join(words)
  sentence = CHINESE_SYMBOLS_AND_PUNCTUATION.remove(sentence)
  sentence = SYMBOLS_AND_PUNCTUATION_EXTENSION.remove(sentence)
  sentence = GENERAL_PUNCTUATION.remove(sentence)
  sentence = sentence.translate({ord('€'): None})
  en_pattern = "[a-zA-Z0-9]+"
  sentence = re.sub(en_pattern, ' ', sentence)
  sentence = '<start> ' + str(sentence) + ' <end>'
  sentence = re.sub(r'[" "]+', " ", sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

Restore the model

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

Evaluate

In [0]:
translate('你叫什么名字')

Input: <start> 你 叫 什么 名字 <end>
Predicted translation: you can delete to eat <end> 


In [0]:
translate('他很棒')

Input: <start> 他 很棒 <end>
Predicted translation: he is wonderful <end> 


In [0]:
translate('请让我唱歌')

Input: <start> 请 让 我 唱歌 <end>
Predicted translation: please let me go home <end> 
